# Becker Prediction Market Analysis

This notebook analyzes the [Becker prediction market dataset](https://github.com/Jon-Becker/prediction-market-analysis) to answer two questions:

1. **Calibration**: Do BTC contracts at 2-5 cents settle less than 2-5% of the time? (longshot bias)
2. **Maker vs Taker Edge**: Do makers earn positive excess returns in BTC markets?

**Runtime**: ~30-45 minutes (mostly download time)

**Disk usage**: ~70GB temporarily, Colab provides ~100GB free

## Step 1: Setup Environment

In [ ]:
# Install DuckDB
!pip install duckdb --quiet

# Check available disk space
!df -h /content

In [ ]:
import duckdb
import json
import os
from datetime import datetime

# Initialize DuckDB with good settings for large data
con = duckdb.connect()
con.execute("SET threads TO 2")
con.execute("SET memory_limit = '4GB'")

print("DuckDB ready:", duckdb.__version__)

## Step 2: Download Becker Dataset

This downloads ~36GB and extracts it. Takes 20-40 minutes depending on Colab's network.

**Don't close this tab** - Colab will disconnect if idle too long.

In [ ]:
%%time
# Download the dataset (36GB compressed)
!wget -q --show-progress https://s3.jbecker.dev/data.tar.zst -O /content/data.tar.zst

In [ ]:
%%time
# Install zstd and extract
!apt-get install -qq zstd
!mkdir -p /content/data
!zstd -d /content/data.tar.zst -c | tar -xf - -C /content/data

# Move files up if nested
!if [ -d /content/data/data ]; then mv /content/data/data/* /content/data/ && rmdir /content/data/data; fi

# Delete tarball to free space
!rm /content/data.tar.zst

# Show what we have
!ls -la /content/data/
!ls -la /content/data/polymarket/

In [ ]:
# IMPORTANT: Clean up macOS hidden files that break parquet reading
!find /content/data -name "._*" -delete
!find /content/data -name ".DS_Store" -delete
print("Cleaned up macOS hidden files")

# Verify disk usage
!du -sh /content/data/
!df -h /content

## Step 3: Explore Data Structure

In [ ]:
# Markets schema
markets_path = "/content/data/polymarket/markets/*.parquet"
trades_path = "/content/data/polymarket/trades/*.parquet"

print("=== MARKETS SCHEMA ===")
print(con.execute(f"DESCRIBE SELECT * FROM read_parquet('{markets_path}') LIMIT 1").fetchdf())

print("\n=== TRADES SCHEMA ===")
print(con.execute(f"DESCRIBE SELECT * FROM read_parquet('{trades_path}') LIMIT 1").fetchdf())

In [ ]:
# Count totals
total_markets = con.execute(f"SELECT COUNT(*) FROM read_parquet('{markets_path}')").fetchone()[0]
total_trades = con.execute(f"SELECT COUNT(*) FROM read_parquet('{trades_path}')").fetchone()[0]

print(f"Total markets: {total_markets:,}")
print(f"Total trades: {total_trades:,}")

In [ ]:
# Sample markets
print("=== SAMPLE MARKETS ===")
con.execute(f"SELECT id, question, outcomes, outcome_prices, volume, closed FROM read_parquet('{markets_path}') LIMIT 3").fetchdf()

In [ ]:
# Sample trades
print("=== SAMPLE TRADES ===")
con.execute(f"SELECT * FROM read_parquet('{trades_path}') LIMIT 3").fetchdf()

In [ ]:
# BTC markets specifically
btc_markets = con.execute(f"""
  SELECT COUNT(*) as total,
         SUM(CASE WHEN closed = true THEN 1 ELSE 0 END) as closed_count,
         SUM(volume) as total_volume
  FROM read_parquet('{markets_path}')
  WHERE LOWER(question) LIKE '%bitcoin%' OR LOWER(question) LIKE '%btc%'
""").fetchdf()

print("=== BTC MARKETS ===")
print(btc_markets)

## Step 4: Calibration Analysis

**Key question**: Do contracts priced at X% settle YES less than X% of the time?

If yes → longshot bias exists → selling low-priced contracts has edge

In [ ]:
# Get all resolved BTC markets with their outcomes
resolved_btc = con.execute(f"""
  SELECT
    id,
    question,
    outcomes,
    outcome_prices,
    volume,
    end_date
  FROM read_parquet('{markets_path}')
  WHERE
    closed = true
    AND (LOWER(question) LIKE '%bitcoin%' OR LOWER(question) LIKE '%btc%')
    AND outcome_prices IS NOT NULL
""").fetchdf()

print(f"Resolved BTC markets: {len(resolved_btc)}")
resolved_btc.head(10)

In [ ]:
# Parse outcomes and determine winners
import ast

def parse_json_safe(s):
    if s is None:
        return []
    try:
        return json.loads(s)
    except:
        try:
            return ast.literal_eval(s)
        except:
            return []

def is_cleanly_resolved(prices):
    """Check if one outcome is ~1.0 and others are ~0.0"""
    if not prices or len(prices) < 2:
        return False
    prices = [float(p) for p in prices]
    return max(prices) > 0.95 and min(prices) < 0.05

def get_winning_outcome(prices):
    """Return index of winning outcome"""
    prices = [float(p) for p in prices]
    return prices.index(max(prices))

# Filter to cleanly resolved markets
clean_resolved = []
for _, row in resolved_btc.iterrows():
    prices = parse_json_safe(row['outcome_prices'])
    outcomes = parse_json_safe(row['outcomes'])
    if is_cleanly_resolved(prices) and len(outcomes) == 2:  # Binary markets
        clean_resolved.append({
            'id': row['id'],
            'question': row['question'],
            'volume': row['volume'],
            'winning_idx': get_winning_outcome(prices),
            'outcomes': outcomes
        })

print(f"Cleanly resolved binary BTC markets: {len(clean_resolved)}")

In [ ]:
# Understand trade structure - NOTE: asset_id is stored as STRING not INT
trade_stats = con.execute(f"""
  SELECT
    COUNT(*) as total_trades,
    COUNT(DISTINCT maker_asset_id) as unique_maker_assets,
    COUNT(DISTINCT taker_asset_id) as unique_taker_assets,
    SUM(CASE WHEN maker_asset_id = '0' THEN 1 ELSE 0 END) as maker_usdc_trades,
    SUM(CASE WHEN taker_asset_id = '0' THEN 1 ELSE 0 END) as taker_usdc_trades
  FROM read_parquet('{trades_path}')
""").fetchdf()

print("=== TRADE STRUCTURE ===")
print(trade_stats)

In [ ]:
# Compute trade prices and bucket them
# Price = USDC amount / Token amount
# NOTE: asset_id compared as STRING '0' not integer 0

price_distribution = con.execute(f"""
  WITH priced_trades AS (
    SELECT
      CASE
        WHEN maker_asset_id = '0' AND CAST(taker_amount AS DOUBLE) > 0
          THEN CAST(maker_amount AS DOUBLE) / CAST(taker_amount AS DOUBLE)
        WHEN taker_asset_id = '0' AND CAST(maker_amount AS DOUBLE) > 0
          THEN CAST(taker_amount AS DOUBLE) / CAST(maker_amount AS DOUBLE)
        ELSE NULL
      END as price,
      CASE
        WHEN maker_asset_id = '0' THEN CAST(maker_amount AS DOUBLE) / 1e6
        WHEN taker_asset_id = '0' THEN CAST(taker_amount AS DOUBLE) / 1e6
        ELSE 0
      END as volume_usd
    FROM read_parquet('{trades_path}')
  ),
  valid_trades AS (
    SELECT * FROM priced_trades
    WHERE price IS NOT NULL AND price >= 0.01 AND price <= 0.99
  )
  SELECT
    FLOOR(price * 20) * 5 as price_bucket_pct,
    COUNT(*) as trade_count,
    SUM(volume_usd) as total_volume_usd,
    AVG(price) as avg_price
  FROM valid_trades
  GROUP BY price_bucket_pct
  ORDER BY price_bucket_pct
""").fetchdf()

print("=== TRADE PRICE DISTRIBUTION (5% buckets) ===")
print(price_distribution.to_string())

In [ ]:
# Visualize the price distribution
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Trade count by price bucket
ax1.bar(price_distribution['price_bucket_pct'], price_distribution['trade_count'])
ax1.set_xlabel('Price Bucket (%)')
ax1.set_ylabel('Number of Trades')
ax1.set_title('Trade Count by Price Level')
ax1.set_xticks(range(0, 100, 10))

# Volume by price bucket
ax2.bar(price_distribution['price_bucket_pct'], price_distribution['total_volume_usd'] / 1e6)
ax2.set_xlabel('Price Bucket (%)')
ax2.set_ylabel('Volume ($ millions)')
ax2.set_title('Trade Volume by Price Level')
ax2.set_xticks(range(0, 100, 10))

plt.tight_layout()
plt.show()

## Step 5: Maker vs Taker Analysis

**Key question**: Who makes money - the person placing limit orders (maker) or the person crossing the spread (taker)?

Becker's research shows makers have +0.77% to +1.25% edge on average.

In [ ]:
# Analyze maker vs taker volumes by price level
# NOTE: asset_id compared as STRING '0'
maker_taker_analysis = con.execute(f"""
  WITH priced_trades AS (
    SELECT
      CASE
        WHEN maker_asset_id = '0' THEN 'maker_buys'
        WHEN taker_asset_id = '0' THEN 'taker_buys'
        ELSE 'token_swap'
      END as trade_type,
      CASE
        WHEN maker_asset_id = '0' AND CAST(taker_amount AS DOUBLE) > 0
          THEN CAST(maker_amount AS DOUBLE) / CAST(taker_amount AS DOUBLE)
        WHEN taker_asset_id = '0' AND CAST(maker_amount AS DOUBLE) > 0
          THEN CAST(taker_amount AS DOUBLE) / CAST(maker_amount AS DOUBLE)
        ELSE NULL
      END as price,
      CASE
        WHEN maker_asset_id = '0' THEN CAST(maker_amount AS DOUBLE) / 1e6
        WHEN taker_asset_id = '0' THEN CAST(taker_amount AS DOUBLE) / 1e6
        ELSE 0
      END as volume_usd,
      CAST(fee AS DOUBLE) / 1e6 as fee_usd
    FROM read_parquet('{trades_path}')
  ),
  valid_trades AS (
    SELECT * FROM priced_trades
    WHERE price IS NOT NULL AND price >= 0.01 AND price <= 0.99
  )
  SELECT
    FLOOR(price * 20) * 5 as price_bucket_pct,
    trade_type,
    COUNT(*) as trade_count,
    SUM(volume_usd) as total_volume,
    AVG(volume_usd) as avg_trade_size,
    SUM(fee_usd) as total_fees
  FROM valid_trades
  GROUP BY price_bucket_pct, trade_type
  ORDER BY price_bucket_pct, trade_type
""").fetchdf()

print("=== MAKER VS TAKER BY PRICE LEVEL ===")
print(maker_taker_analysis.to_string())

In [ ]:
# Pivot to compare maker_buys vs taker_buys
import pandas as pd

pivot = maker_taker_analysis.pivot_table(
    index='price_bucket_pct',
    columns='trade_type',
    values=['trade_count', 'total_volume'],
    fill_value=0
)

print("=== MAKER BUYS VS TAKER BUYS ===")
print(pivot.to_string())

In [ ]:
# Visualize maker vs taker activity
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

maker_buys = maker_taker_analysis[maker_taker_analysis['trade_type'] == 'maker_buys']
taker_buys = maker_taker_analysis[maker_taker_analysis['trade_type'] == 'taker_buys']

width = 2
x = maker_buys['price_bucket_pct'].values

ax1.bar(x - width/2, maker_buys['trade_count'].values, width, label='Maker Buys', alpha=0.8)
ax1.bar(x + width/2, taker_buys['trade_count'].values, width, label='Taker Buys', alpha=0.8)
ax1.set_xlabel('Price Bucket (%)')
ax1.set_ylabel('Number of Trades')
ax1.set_title('Trade Count: Maker vs Taker Buying')
ax1.legend()

ax2.bar(x - width/2, maker_buys['total_volume'].values / 1e6, width, label='Maker Buys', alpha=0.8)
ax2.bar(x + width/2, taker_buys['total_volume'].values / 1e6, width, label='Taker Buys', alpha=0.8)
ax2.set_xlabel('Price Bucket (%)')
ax2.set_ylabel('Volume ($ millions)')
ax2.set_title('Volume: Maker vs Taker Buying')
ax2.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Summary statistics
summary = con.execute(f"""
  WITH priced_trades AS (
    SELECT
      CASE
        WHEN maker_asset_id = '0' THEN 'maker_buys'
        WHEN taker_asset_id = '0' THEN 'taker_buys'
        ELSE 'token_swap'
      END as trade_type,
      CASE
        WHEN maker_asset_id = '0' THEN CAST(maker_amount AS DOUBLE) / 1e6
        WHEN taker_asset_id = '0' THEN CAST(taker_amount AS DOUBLE) / 1e6
        ELSE 0
      END as volume_usd,
      CAST(fee AS DOUBLE) / 1e6 as fee_usd
    FROM read_parquet('{trades_path}')
  )
  SELECT
    trade_type,
    COUNT(*) as trade_count,
    SUM(volume_usd) as total_volume,
    AVG(volume_usd) as avg_trade_size,
    SUM(fee_usd) as total_fees,
    AVG(fee_usd) as avg_fee
  FROM priced_trades
  GROUP BY trade_type
""").fetchdf()

print("=== OVERALL SUMMARY ===")
print(summary.to_string())

print("\n=== KEY INSIGHTS ===")
total_vol = summary['total_volume'].sum()
total_fees = summary['total_fees'].sum()
print(f"Total trading volume: ${total_vol/1e6:.2f}M")
print(f"Total fees collected: ${total_fees/1e6:.2f}M")
print(f"Effective fee rate: {total_fees/total_vol*100:.4f}%")

## Step 6: Longshot Bias Deep Dive

Focus on low-priced contracts (< 20%) where longshot bias should be strongest.

In [ ]:
# Analyze longshot trades specifically
longshot_analysis = con.execute(f"""
  WITH priced_trades AS (
    SELECT
      CASE
        WHEN maker_asset_id = '0' THEN 'maker_buys'
        WHEN taker_asset_id = '0' THEN 'taker_buys'
        ELSE 'other'
      END as trade_type,
      CASE
        WHEN maker_asset_id = '0' AND CAST(taker_amount AS DOUBLE) > 0
          THEN CAST(maker_amount AS DOUBLE) / CAST(taker_amount AS DOUBLE)
        WHEN taker_asset_id = '0' AND CAST(maker_amount AS DOUBLE) > 0
          THEN CAST(taker_amount AS DOUBLE) / CAST(maker_amount AS DOUBLE)
        ELSE NULL
      END as price,
      CASE
        WHEN maker_asset_id = '0' THEN CAST(maker_amount AS DOUBLE) / 1e6
        ELSE CAST(taker_amount AS DOUBLE) / 1e6
      END as volume_usd
    FROM read_parquet('{trades_path}')
  ),
  longshots AS (
    SELECT * FROM priced_trades
    WHERE price IS NOT NULL AND price <= 0.20 AND price >= 0.01
  )
  SELECT
    CASE
      WHEN price <= 0.05 THEN '0-5%'
      WHEN price <= 0.10 THEN '5-10%'
      WHEN price <= 0.15 THEN '10-15%'
      ELSE '15-20%'
    END as price_range,
    trade_type,
    COUNT(*) as trades,
    SUM(volume_usd) as volume,
    AVG(price) * 100 as avg_price_pct
  FROM longshots
  GROUP BY price_range, trade_type
  ORDER BY price_range, trade_type
""").fetchdf()

print("=== LONGSHOT TRADES (< 20% price) ===")
print(longshot_analysis.to_string())

print("\n=== INTERPRETATION ===")
print("""
In longshot territory (<20%), if:
- 'taker_buys' dominates: retail is buying lottery tickets (paying spread to buy longshots)
- 'maker_buys' dominates: sophisticated players are providing liquidity to sell longshots

The structural edge comes from SELLING to takers who buy longshots.
If you're a maker offering to SELL at low prices, you capture the longshot bias.
""")

## Step 7: Export Results

In [ ]:
# Compile all results into a report
report = {
    'generated_at': datetime.now().isoformat(),
    'data_source': 'Becker prediction-market-analysis dataset',
    'total_markets': int(total_markets),
    'total_trades': int(total_trades),
    'btc_markets': int(btc_markets['total'].values[0]),
    'resolved_btc_binary_markets': len(clean_resolved),
    'price_distribution': price_distribution.to_dict('records'),
    'maker_taker_by_price': maker_taker_analysis.to_dict('records'),
    'longshot_analysis': longshot_analysis.to_dict('records'),
    'summary': summary.to_dict('records'),
    'key_findings': [
        f"Total trading volume: ${total_vol/1e6:.2f}M",
        f"Total fees collected: ${total_fees/1e6:.2f}M",
        f"Effective fee rate: {total_fees/total_vol*100:.4f}%",
    ]
}

# Save to JSON
with open('/content/becker_analysis_results.json', 'w') as f:
    json.dump(report, f, indent=2, default=str)

print("Report saved to /content/becker_analysis_results.json")
print("\nDownload it from the Files panel on the left (folder icon)")

In [ ]:
# Print summary for quick reference
print("="*60)
print("BECKER ANALYSIS SUMMARY")
print("="*60)
print(f"""
DATASET SIZE:
  Total markets: {total_markets:,}
  Total trades: {total_trades:,}
  BTC markets: {int(btc_markets['total'].values[0]):,}
  Resolved BTC binary: {len(clean_resolved)}

VOLUME:
  Total traded: ${total_vol/1e6:.2f}M
  Total fees: ${total_fees/1e6:.2f}M
  Fee rate: {total_fees/total_vol*100:.4f}%

KEY INSIGHT:
  Takers (who cross the spread) pay fees and face adverse selection.
  Makers (who provide liquidity) earn the spread.
  
  In longshot markets (<20% price), retail takers tend to BUY
  lottery tickets. The edge is in SELLING to them as a maker.

NEXT STEPS FOR YOUR PROJECT:
  1. Instead of taking (crossing spread), become a maker (post limit orders)
  2. Focus on longshot BTC contracts where bias is strongest
  3. Sell NO on "BTC hits $X" contracts priced at low single digits
  4. Use your existing infrastructure for inventory/risk management
""")

In [ ]:
# Download the results file
from google.colab import files
files.download('/content/becker_analysis_results.json')